In [1]:
#based on "Let's build GPT: from scratch, in code, spelled out." from Andrej Karpathy

# download Quo Vadis from Henryk Sienkiewicz
!wget "http://biblioteka.kijowski.pl/sienkiewicz%20henryk/quo%20vadis%20(ang).txt"
!mv "quo vadis (ang).txt" quo_vadis.txt

--2024-05-16 06:33:52--  http://biblioteka.kijowski.pl/sienkiewicz%20henryk/quo%20vadis%20(ang).txt
Resolving biblioteka.kijowski.pl (biblioteka.kijowski.pl)... 77.65.215.2
Connecting to biblioteka.kijowski.pl (biblioteka.kijowski.pl)|77.65.215.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642072 (627K) [text/plain]
Saving to: ‘quo vadis (ang).txt’

quo vadis (ang).txt 100%[===================>] 627,02K  1,59MB/s    in 0,4s    

2024-05-16 06:33:53 (1,59 MB/s) - ‘quo vadis (ang).txt’ saved [642072/642072]



In [2]:
#get pointer to read text
with open('quo_vadis.txt', encoding='latin-1') as f:
    text = f.read()

In [3]:
print('len of text: {}'.format(len(text)))
print(text[0:1000])

len of text: 630169
Project Gutenberg EBook, Quo Vadis, by Henryk Sienkiewicz

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg, and how to get involved.


**Welcome To The World of Free Plain Vanilla Electronic Texts**

**EBooks Readable By Both Humans and By Computers, Since 1971**

*****These EBooks Were Prepared By Thousands of Volunteers*****



Title: Quo Vadis A Narrative of th

In [4]:
#get all unique charaters that occure in text
chars = sorted(list(set(text)))
print(chars)
vocab_size = len(chars)
print("vocabulary length: {}".format(len(chars)))

['\n', ' ', '!', '"', '#', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x8c', '\x9c', 'Æ', 'æ', 'ç', 'é', 'ë', 'ô']
vocabulary length: 86


In [5]:
# transform characters to numeric data
strToInt = {ch:i for i,ch in enumerate(chars)}
intToStr = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[strToInt[c] for c in s] #encode string to integers
decode = lambda l:''.join([intToStr[i] for i in l]) #decode intigers to string

encoded = encode('quo vadis domine')
print('Encoded:', encoded)
print('Decoded:', decode(encoded))

Encoded: [68, 72, 66, 1, 73, 52, 55, 60, 70, 1, 55, 66, 64, 60, 65, 56]
Decoded: quo vadis domine


In [6]:
#encoding entire "quo vadis" text
import torch #PyTorch lib is used for building transformer components https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long) #encode whole text
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([630169]) torch.int64
tensor([39, 69, 66, 61, 56, 54, 71,  1, 30, 72, 71, 56, 65, 53, 56, 69, 58,  1,
        28, 25, 66, 66, 62,  9,  1, 40, 72, 66,  1, 45, 52, 55, 60, 70,  9,  1,
        53, 76,  1, 31, 56, 65, 69, 76, 62,  1, 42, 60, 56, 65, 62, 60, 56, 74,
        60, 54, 77,  0,  0, 26, 66, 67, 76, 69, 60, 58, 59, 71,  1, 63, 52, 74,
        70,  1, 52, 69, 56,  1, 54, 59, 52, 65, 58, 60, 65, 58,  1, 52, 63, 63,
         1, 66, 73, 56, 69,  1, 71, 59, 56,  1, 74, 66, 69, 63, 55, 11,  1, 25,
        56,  1, 70, 72, 69, 56,  1, 71, 66,  1, 54, 59, 56, 54, 62,  1, 71, 59,
        56,  0, 54, 66, 67, 76, 69, 60, 58, 59, 71,  1, 63, 52, 74, 70,  1, 57,
        66, 69,  1, 76, 66, 72, 69,  1, 54, 66, 72, 65, 71, 69, 76,  1, 53, 56,
        57, 66, 69, 56,  1, 55, 66, 74, 65, 63, 66, 52, 55, 60, 65, 58,  1, 66,
        69,  1, 69, 56, 55, 60, 70, 71, 69, 60, 53, 72, 71, 60, 65, 58,  0, 71,
        59, 60, 70,  1, 66, 69,  1, 52, 65, 76,  1, 66, 71, 59, 56, 69,  1, 39,
       

In [7]:
#split data into train and validation sets
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size + 1]

tensor([39, 69, 66, 61, 56, 54, 71,  1, 30])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([39]) the target: 69
when input is tensor([39, 69]) the target: 66
when input is tensor([39, 69, 66]) the target: 61
when input is tensor([39, 69, 66, 61]) the target: 56
when input is tensor([39, 69, 66, 61, 56]) the target: 54
when input is tensor([39, 69, 66, 61, 56, 54]) the target: 71
when input is tensor([39, 69, 66, 61, 56, 54, 71]) the target: 1
when input is tensor([39, 69, 66, 61, 56, 54, 71,  1]) the target: 30


In [10]:
torch.manual_seed(2527)
batch_size = 4 # number of independant sequences processed in parallel
block_size = 8 # maximum context length of predictions

def get_batch(split):
    # generate batch of data of input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[ 1, 81, 58, 60, 70,  1, 66, 57],
        [58, 66,  0, 53, 56, 76, 66, 65],
        [60, 70,  1, 52, 69, 64,  9,  0],
        [64, 52, 60, 55, 56, 65, 70,  1]])
targets:
torch.Size([4, 8])
tensor([[81, 58, 60, 70,  1, 66, 57,  1],
        [66,  0, 53, 56, 76, 66, 65, 55],
        [70,  1, 52, 69, 64,  9,  0, 52],
        [52, 60, 55, 56, 65, 70,  1, 60]])
----
when input is [1] the target: 81
when input is [1, 81] the target: 58
when input is [1, 81, 58] the target: 60
when input is [1, 81, 58, 60] the target: 70
when input is [1, 81, 58, 60, 70] the target: 1
when input is [1, 81, 58, 60, 70, 1] the target: 66
when input is [1, 81, 58, 60, 70, 1, 66] the target: 57
when input is [1, 81, 58, 60, 70, 1, 66, 57] the target: 1
when input is [58] the target: 66
when input is [58, 66] the target: 0
when input is [58, 66, 0] the target: 53
when input is [58, 66, 0, 53] the target: 56
when input is [58, 66, 0, 53, 56] the target: 76
when input is [58, 66, 0,

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(2334)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        #idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus on last time step
            logits = logits[:, -1, :] #becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # samble from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 86])
tensor(4.8234, grad_fn=<NllLossBackward0>)

JK5kéDU!7ë ô*6wQ)8frZÆ9oBhzdZ?]1:mZxAsb1l tor.RX (8rj6O;ç8ÆTPAb);HIôIU:;*]wFRt-jU6tdYzô5,yP-*ëFH


In [12]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
                             

In [13]:
batch_size = 32
for steps in range(10000):
    #sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3486311435699463


In [14]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


veces. jondiriu, ce ps, th oure a are[pe; t, m t il mbe ws thickndeth s wad
uert ctherewasanthe, g Afelveng foxcethiri! if The po he icaty ha[M12Ly.  te whed t isthiste ncked n t d tiang  asthigerey irnd to Ahd ha d d(, f has o her gim tulereg towhais wong t, tk; or e;
" atmROtununazedearinthorouessseroinchewofugas ns Thyok?"
stheto had ad wium, wad, theppinqus ia wr wowo t bl m. iushe  hineribus


In [16]:
torch.manual_seed(1234)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #t,C
        xbow[b,t] = torch.mean(xprev, 0)

In [30]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x #(B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [20]:
x[0]

tensor([[-0.1117, -0.4966],
        [ 0.1631, -0.8817],
        [ 0.0539,  0.6684],
        [-0.0597, -0.4675],
        [-0.2153,  0.8840],
        [-0.7584, -0.3689],
        [-0.3424, -1.4020],
        [ 0.3206, -1.0219]])

In [21]:
xbow[0]

tensor([[-0.1117, -0.4966],
        [ 0.0257, -0.6891],
        [ 0.0351, -0.2366],
        [ 0.0114, -0.2944],
        [-0.0339, -0.0587],
        [-0.1547, -0.1104],
        [-0.1815, -0.2949],
        [-0.1187, -0.3858]])

In [25]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [27]:
torch.manual_seed(23)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(f'a={a}')
print('--')
print(f'b={b}')
print('--')
print(f'c={c}')
print('--')

a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=tensor([[1., 6.],
        [6., 7.],
        [0., 2.]])
--
c=tensor([[1.0000, 6.0000],
        [3.5000, 6.5000],
        [2.3333, 5.0000]])
--
